In [1]:
"""
不含模型参数的自定义层
"""
from mxnet import gluon, nd
from mxnet.gluon import nn

#  继承Block类，重写init, forward.
class CenteredLayer(nn.Block):
    def __init__(self):
        super(CenteredLayer, self).__init__()

    def forward(self, x):
        return x - x.mean()


layer = CenteredLayer()
layer(nd.array([1, 2, 3, 4, 5]))


[-2. -1.  0.  1.  2.]
<NDArray 5 @cpu(0)>

In [2]:
"""
用它来构造更复杂的模型
"""
net = nn.Sequential()
net.add(nn.Dense(128),
        CenteredLayer())

In [5]:
#  打印自定义层各个输出的均值
net.initialize()
y = net(nd.random.uniform(shape=(4,8)))
y.mean().asscalar()

4.8748916e-10

In [6]:
"""
含模型参数的自定义层
"""
params = gluon.ParameterDict()
params.get('params', shape=(2, 3))
params

(
  Parameter params (shape=(2, 3), dtype=<class 'numpy.float32'>)
)

In [7]:
"""
以下实现一个含权重和偏差参数的全连接层。

"""
class MyDense(nn.Block):
    def __init__(self, units, in_units):
        super(MyDense, self).__init__()
        self.weight = self.params.get('weight', shape=(in_units, units))
        self.bias = self.params.get('bias', shape=(units, ))
    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + self.bias.data()
        return nd.relu(linear)

"""
实例化MyDense类并访问它的模型参数。

"""
dense = MyDense(units=3, in_units=5)
dense.params

mydense0_ (
  Parameter mydense0_weight (shape=(5, 3), dtype=<class 'numpy.float32'>)
  Parameter mydense0_bias (shape=(3,), dtype=<class 'numpy.float32'>)
)

In [8]:
"""
直接使用自定义层做前向计算。
"""
dense.initialize()
dense(nd.random.uniform(shape=(2, 5)))


[[0.06277736 0.08472438 0.        ]
 [0.10200627 0.0789248  0.        ]]
<NDArray 2x3 @cpu(0)>

In [9]:
"""
使用自定义层构造模型
"""
net = nn.Sequential()
net.add(MyDense(8, in_units=64),
        MyDense(1, in_units=8))
net.initialize()
net(nd.random.uniform(shape=(2, 64)))


[[0.01334638]
 [0.00830848]]
<NDArray 2x1 @cpu(0)>